# Goal 
- Given a dataset with 50 features we must reduce it to `7` features including the 2 possible target variables `A1Cresult` and `readmitted`. We must then find `5` more features that can have strong predictive power in determining our desired target. We decided that `7` is a good number because it greatly reducing the number of potential patterns to be discovered without losing information. If we use all 50 it basically will be almost impossible to find the true pattern.

# About the dataset
- The dataset represents ten years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. Each row concerns hospital records of patients diagnosed with diabetes, who underwent laboratory, medications, and stayed up to 14 days. The goal is to determine the early readmission of the patient within 30 days of discharge. The problem is important for the following reasons. Despite high-quality evidence showing improved clinical outcomes for diabetic patients who receive various preventive and therapeutic interventions, many patients do not receive them. This can be partially attributed to arbitrary diabetes management in hospital environments, which fail to attend to glycemic control. Failure to provide proper diabetes care not only increases the managing costs for the hospitals (as the patients are readmitted) but also impacts the morbidity and mortality of the patients, who may face complications associated with diabetes.

### Import Libraries & Data #### 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
print("Libraries imported ....")

Libraries imported ....


In [2]:
# Load Data
df = pd.read_csv("./diabetic_data.csv")
print("data loaded ...")

data loaded ...


In [3]:
df.replace('?', np.nan, inplace=True)   # Replace all '?' to NaN for easier processing
print("? -> NaN")

? -> NaN


In [4]:
df.head(5)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
df.shape

(101766, 50)

# Features to Keep 

### Potential Targets
1. `readmitted`  - This represents whether the patient was readmitted and is directly related to patient outcomes.

2. `A1Cresult` - Outcome of the hemoglobin A1c test, which measures the average blood sugar level over the past 2-3 months. It's a key metric for assessing diabetes control.

### Additional 5 Features 
- We've determined that these features have the strongest predictive power on determining if a patient has diabetes. It will give a good glimpse of who the patient is, the severity of their diagnosis, and the complexity of their overall case.

3. `age` - Despite age being in a range we believe that age still has strong predictive power on determining 'readmitted' or 'A1Cresult' as the older the person is the more prone to having diabetes.

4. `time_in_hospital` - Indicates the length of hospital stay, potentially related to both readmission likelihood and diabetes control. 


5. `num_medications` - The number of medications prescribed could reflect the severity and complexity of the patient's condition.

6. `number_diagnoses` - represents the total number of diagnoses, indicating the complexity of a patients condition.

7. `insulin` - a key feature on indicating if someone has diabetes is if they are taking insulin as the prescription is cruicial in diabetes management. This maybe one of the most important features if we were to guess.


### Total features = `7`
- It also is great that these features have no null values which makes ensures data integrity as this is the true values of the dataset and we won't have to rely on imputation that makes the model less capable of generalizing as the imputation methods usually rely on interpolation.



In [9]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [15]:
features_to_keep = ['age', 'time_in_hospital', 'num_medications', 'number_diagnoses', 'insulin', 'A1Cresult', 'readmitted']
print(f"Features to keep: {features_to_keep}. Total number of features: {len(features_to_keep)}")

Features to keep: ['age', 'time_in_hospital', 'num_medications', 'number_diagnoses', 'insulin', 'A1Cresult', 'readmitted']. Total number of features: 7


In [16]:
# Reduce dataset
reduced_df = df[features_to_keep]
print("Dataframe Reduced ...")

Dataframe Reduced ...


In [17]:
reduced_df

,age,time_in_hospital,num_medications,number_diagnoses,insulin,A1Cresult,readmitted
0,[0-10),1,1,1,No,NaN,NO
1,[10-20),3,18,9,Up,NaN,>30
2,[20-30),2,13,6,No,NaN,NO
3,[30-40),2,16,7,Up,NaN,NO
4,[40-50),1,8,5,Steady,NaN,NO
...,...,...,...,...,...,...,...
101761,[70-80),3,16,9,Down,>8,>30
101762,[80-90),5,18,9,Steady,NaN,NO
101763,[70-80),1,9,13,Down,NaN,NO
101764,[80-90),10,21,9,Up,NaN,NO


In [18]:
# Generate a null values report
null_report = pd.DataFrame({
    'Total Nulls': reduced_df.isnull().sum(),
    'Percentage Null': (reduced_df.isnull().mean() * 100)
}).sort_values(by='Total Nulls', ascending=False)

print(null_report)


                  Total Nulls  Percentage Null
A1Cresult               84748        83.277322
age                         0         0.000000
time_in_hospital            0         0.000000
num_medications             0         0.000000
number_diagnoses            0         0.000000
insulin                     0         0.000000
readmitted                  0         0.000000


This is great, we have no null values. A1Cresult seems like there are a lot of null values but that is intentional as having no result means that maybe the test was not even ordered as the case may not be as serious. 

### Feature transformation

## age <br>
Possible Values: 10 possible values. The ages aren't specific and have been turned into bins. We will convert them into numerical bins as it would be easier to process these values computationally if they were integers. <br>
- [0-10)  -> 1
- [10-20)  -> 2
- [20-30) -> 3
- [30-40) -> 4
- [40-50) -> 5
- [50-60) -> 6
- [60-70) -> 7
- [70-80) -> 8
- [80-90) -> 9
- [90-100) -> 10

In [21]:
reduced_df['age'].unique()

array(['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)',
       '[60-70)', '[70-80)', '[80-90)', '[90-100)'], dtype=object)

In [ ]:
# Define the mapping from age bins to integers
age_mapping = {
    '[0-10)': 1,
    '[10-20)': 2,
    '[20-30)': 3,
    '[30-40)': 4,
    '[40-50)': 5,
    '[50-60)': 6,
    '[60-70)': 7,
    '[70-80)': 8,
    '[80-90)': 9,
    '[90-100)': 10
}

# Replace age bins with integers in the DataFrame
reduced_df['age'] = reduced_df['age'].replace(age_mapping)

0    1
1    2
2    3
3    4
4    5
Name: age, dtype: int64


C:\Users\usflo\AppData\Local\Temp\ipykernel_14684\3357611460.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df['age'] = reduced_df['age'].replace(age_mapping)


In [24]:
reduced_df['age'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

## time_in_hospital <br>
- there is no need to transform this feature as it is a integer which is just the number of days between admission and discharge. It also has no null values

## num_medications <br>
- like `time_in_hospital` this does not need to be transformed as it is an integer indicating the nubmer of medications. 

## number_diagnoses <br>
- similar to the previous two features this is just a count of medications which does not need to be transformed

### Why didn't we transform the previous 3?  <br>
- These are just counts and if we transformed them it would just be bin the values into the unique values which will make our data untrue. There is no reason to bin numerical counts

## insulin <br>

- `No` - No insulin was prescribed or administered. 
- `Down` - Indicates a decrease in the insulin dosage.
- `Steady` - Indicates that the insulin dosage remained consistent.
- `Up` - Indicates an increase in the insulin dosage.
<br>
-  Modification we convert these into numerical bins as well with the mapping them into integers.

array(['No', 'Up', 'Steady', 'Down'], dtype=object)

In [33]:
# Define the mapping for insulin
insulin_mapping = {
    'No': 1,      # Map 'No' to 1
    'Down': 2,    # Map 'Down' to 2
    'Steady': 3,  # Map 'Steady' to 3
    'Up': 4       # Map 'Up' to 4
}

# Replace insulin values with the mapping in the DataFrame
reduced_df['insulin'] = reduced_df['insulin'].replace(insulin_mapping)

print("Insulin has been mapped ...")

Insulin has been mapped ...


C:\Users\usflo\AppData\Local\Temp\ipykernel_14684\2480410216.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df['insulin'] = reduced_df['insulin'].replace(insulin_mapping)


In [34]:
reduced_df['insulin'].unique()

array([1, 4, 3, 2], dtype=int64)

## A1Cresult <br>
- `>8` : This suggests the patient has consistently high blood glucose levels over time and may require adjustments to their diabetes management plan, such as changes to medication, diet, or lifestyle.

- `>7` : This level suggests the patient is managing their blood sugar better than >8 but still may not meet the recommended target range for A1c..

- `Norm` : The patient has good blood sugar control, likely meeting the target set by their healthcare provider.

- `nan` : This means the test wasn't conducted or results are unavailable, so no inference about long-term blood sugar control can be made. This may imply that the tests were not needed making the case less complicated.


- The modification will be to convert the unique values into numerical values so that we can process the data easier.

In [28]:
reduced_df['A1Cresult'].unique()

array([nan, '>7', '>8', 'Norm'], dtype=object)

In [29]:
# Define the mapping for A1Cresult
a1c_mapping = {
    np.nan: 1,  # Map NaN to 1
    'Norm': 2,  # Map 'Norm' to 2
    '>7': 3,    # Map '>7' to 3
    '>8': 4     # Map '>8' to 4
}

# Replace A1Cresult values with the mapping in the DataFrame
reduced_df['A1Cresult'] = reduced_df['A1Cresult'].replace(a1c_mapping)
print("Mapped A1Cresult ...")

Mapped A1Cresult ...


C:\Users\usflo\AppData\Local\Temp\ipykernel_14684\1117270933.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df['A1Cresult'] = reduced_df['A1Cresult'].replace(a1c_mapping)


In [30]:
reduced_df['A1Cresult'].unique()

array([1, 3, 4, 2], dtype=int64)

## readmitted <br>
- NO - patient has not been readmitted back within the time of the study
- `>30` - Patient was readmitted after a month
- `<30` - Patient was readmitted within a month
<br>

- similar to what we've done before we just map this into numerical integers as well

In [ ]:
reduced_df['readmitted'].unique()

array(['NO', '>30', '<30'], dtype=object)

In [37]:
# Define the mapping for readmitted
readmitted_mapping = {
    'NO': 1,     # Map 'NO' to 1
    '>30': 2,    # Map '>30' to 2
    '<30': 3     # Map '<30' to 3
}

# Replace readmitted values with the mapping in the DataFrame
reduced_df['readmitted'] = reduced_df['readmitted'].replace(readmitted_mapping)

print("readmitted has been mapped ...")


readmitted has been mapped ...


C:\Users\usflo\AppData\Local\Temp\ipykernel_14684\2274955807.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df['readmitted'] = reduced_df['readmitted'].replace(readmitted_mapping)


In [38]:
reduced_df['readmitted'].unique()

array([1, 2, 3], dtype=int64)

# Train-test-split
- Our data has been fully preprocessed and transformed and is ready to be used for modeling. To turn this into a supervised learning task we must assing X (our features) and Y(our target) and split the data. We are opting in for a 80-train, 20-test, split or the default as there are `101,766` rows which is plenty to train on to evaluate on.

In [39]:
reduced_df.head(10)

,age,time_in_hospital,num_medications,number_diagnoses,insulin,A1Cresult,readmitted
0,1,1,1,1,1,1,1
1,2,3,18,9,4,1,2
2,3,2,13,6,1,1,1
3,4,2,16,7,4,1,1
4,5,1,8,5,3,1,1
5,6,3,16,9,3,1,2
6,7,4,21,7,3,1,1
7,8,5,12,8,1,1,2
8,9,13,28,8,3,1,1
9,10,12,18,8,3,1,1


In [41]:
# Export full dataset
reduced_df.to_csv('transformed_diabetic_data.csv', index=False)
print("dataset exported ...")

dataset exported ...


In [42]:
# Separate features (X) and target labels (y)
X = reduced_df.drop(columns=['readmitted'])  # Features
y = reduced_df['readmitted']  # Target labels
print("X and y separated ...")

X and y separated ...


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train test split executed ...")

Train test split executed ...


In [44]:
# Combine the training data and labels
train_data = X_train.copy()
train_data['readmitted'] = y_train

# Combine the testing data and labels
test_data = X_test.copy()
test_data['readmitted'] = y_test

# Export training data to a CSV file
train_data.to_csv('transformed_diabetic_data_train.csv', index=False)

# Export testing data to a CSV file
test_data.to_csv('transformed_diabetic_data_test.csv', index=False)

print("Training and Testing Data exported ...")

Training and Testing Data exported ...
